# Интеллектуальный анализ данных – весна 2024
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [1]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [2]:
df = pd.read_csv('/content/sample_data/tweets_coronavirus.csv', encoding='latin-1')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная эмоциональная окраска и 0 - если отрицательная.

In [3]:
df['Sentiment'].unique()

array(['Positive', 'Extremely Negative', 'Negative', 'Extremely Positive'],
      dtype=object)

In [4]:
df_changed = df.copy()
df_changed['Sentiment'] = [1 if 'Positive' in el else 0 for el in df['Sentiment']]
df_changed.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Сбалансированы ли классы?

In [5]:
df_changed['Sentiment'].value_counts()

Sentiment
1    18046
0    15398
Name: count, dtype: int64

**Ответ:** скорее да, чем нет, так как перевес положительного класса составляет порядка 3000 элементов, что в относительной шкале равняется примерно 7-8%

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [6]:
df_changed.isna().sum()

UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [7]:
df_filled = df_changed.fillna(value='Unknown')
df_filled.isna().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

Разделите данные на обучающие и тестовые в соотношении 7 : 3 и `random_state=0`

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_filled, test_size=0.3, random_state=0, shuffle=True, stratify=df_filled['Sentiment'])
train

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
20012,28197,73149,Unknown,26-03-2020,"? New Podcast! ""Businesses Answering the COVID...",1
28328,38537,83489,"Whangarei, New Zealand",08-04-2020,Covid 19 coronavirus: Hundreds request virus c...,1
16192,23512,68464,Unknown,24-03-2020,@Crohnoid @ChachyOwen @IBDPassport @J9JSM @vik...,1
31977,43130,88082,Canada ????? Edmonton,12-04-2020,@irSkullBeard @aseip1 Are social determinants ...,0
11431,17656,62608,"Butte, Montana",21-03-2020,As factories are stilled around the globe beca...,0
...,...,...,...,...,...,...
12017,18377,63329,Eastern WA,21-03-2020,Now I don't feel so silly about panic planting...,0
4517,9295,54247,"Dallas, Texas 75251",19-03-2020,hmm.. @CDCgov would it be a good idea to start...,1
16140,23451,68403,Unknown,24-03-2020,Selling surgical masks.\r\r\nReal surgical mas...,1
21705,30284,75236,Unknown,02-04-2020,Just spoke to two despots and they agreed to r...,1


## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведение токенов в нижний регистр

In [9]:
from nltk import tokenize
merged_text = ' '.join(df_filled['OriginalTweet'].tolist()).lower()
tok = tokenize.WhitespaceTokenizer()
tokens = tok.tokenize(merged_text)
print(tokens[:5000])

['advice', 'talk', 'to', 'your', 'neighbours', 'family', 'to', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'with', 'phone', 'numbers', 'of', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'up', 'online', 'shopping', 'accounts', 'if', 'poss', 'adequate', 'supplies', 'of', 'regular', 'meds', 'but', 'not', 'over', 'order', 'coronavirus', 'australia:', 'woolworths', 'to', 'give', 'elderly,', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'covid-19', 'outbreak', 'https://t.co/binca9vp8p', 'my', 'food', 'stock', 'is', 'not', 'the', 'only', 'one', 'which', 'is', 'empty...', 'please,', "don't", 'panic,', 'there', 'will', 'be', 'enough', 'food', 'for', 'everyone', 'if', 'you', 'do', 'not', 'take', 'more', 'than', 'you', 'need.', 'stay', 'calm,', 'stay', 'safe.', '#covid19france', '#covid_19', '#covid19', '#coronavirus', '#confinement', '#confinementotal', '#confinementgeneral', 'https://t.co/zrlg0z520j', 'me,', 'ready', 'to', 'go', 'at', 'supermarket', 'dur

Какой размер словаря получился?

In [10]:
len(set(tokens))

103200

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе

In [11]:
def print_most_common_10(token_counts: dict):
  """
  Функция для вывода 10 самых популярных токенов словаря.
  """
  for token, count in token_counts.most_common(10):
    print(f'Токен: {token}, количество вхождений: {count}')

from collections import Counter
token_counts = Counter(tokens)
print_most_common_10(token_counts)

Токен: the, количество вхождений: 38250
Токен: to, количество вхождений: 33447
Токен: and, количество вхождений: 20935
Токен: of, количество вхождений: 18578
Токен: a, количество вхождений: 16667
Токен: in, количество вхождений: 16024
Токен: for, количество вхождений: 12193
Токен: #coronavirus, количество вхождений: 11759
Токен: is, количество вхождений: 10596
Токен: are, количество вхождений: 9958


**Ответ:** данные слова являются служебными частями речи (предлоги, союзы, вспомогательные глаголы и так далее), которые, очевидно, в любом языке встречаются чаще всего

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
words = [el.lower() for el in stopwords.words('english')] # на всякий случай приводим к нижнему регистру (например, чтобы учесть стоп-слово I)

print(words)
print()

for el in words:
  del token_counts[el]

print_most_common_10(token_counts)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Ответ:** теперь топ-10 самых популярных слов несет больше смысла, так как мы удалили из кандитатов стоп-слова (самые часто встречающиеся в письменной речи слова)

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [14]:
def least_20_common_tokens(token_counts: dict):
  """
  Функция для вывода 20 самых непопулярных токенов словаря.
  """
  least_common_tokens = sorted(token_counts.items(), key=lambda x: x[1])[:20]

  for token, count in least_common_tokens:
      print(f'Токен: {token}, количество вхождений: {count}')
least_20_common_tokens(token_counts)

Токен: poss, количество вхождений: 1
Токен: australia:, количество вхождений: 1
Токен: https://t.co/binca9vp8p, количество вхождений: 1
Токен: #confinementgeneral, количество вхождений: 1
Токен: https://t.co/zrlg0z520j, количество вхождений: 1
Токен: litteraly, количество вхождений: 1
Токен: #coronavirusfrance, количество вхождений: 1
Токен: https://t.co/usmualq72n, количество вхождений: 1
Токен: @tim_dodson, количество вхождений: 1
Токен: https://t.co/cfxch7a2lu, количество вхождений: 1
Токен: civics, количество вхождений: 1
Токен: about"., количество вхождений: 1
Токен: https://t.co/iefdnehgdo, количество вхождений: 1
Токен: https://t.co/kw91zj5o5i, количество вхождений: 1
Токен: prevention,we, количество вхождений: 1
Токен: 19?., количество вхождений: 1
Токен: #govindia, количество вхождений: 1
Токен: #horningsea, количество вхождений: 1
Токен: https://t.co/lsgrxxhjhh, количество вхождений: 1
Токен: https://t.co/8ywakfjexc, количество вхождений: 1


**Ответ:** данные токены представляют либо ссылки, либо непопулярные хеш-теги, никнеймы, видимо, в телеграмме, а также слова с грамматическими или стилистическими ошибками. Понятное дело, что токены из этих групп будут уникальными (ссылки и никнеймы обязаны быть уникальными, а слова с ошибками или непопулярные хеш-теги вряд ли будут повторены другими пользователями в силу их специфики). Думаю, что эти токены можно удалить, но вряд ли это существенно повлияет на итоговый результат (но это можно провести в качестве отдельного эксперимента, например)



Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [15]:
from nltk.tokenize import TweetTokenizer
tw = TweetTokenizer()
tokens = tw.tokenize(merged_text)
token_counts = Counter(tokens)

print_most_common_10(token_counts)

Токен: the, количество вхождений: 38499
Токен: ., количество вхождений: 34264
Токен: to, количество вхождений: 33588
Токен: ,, количество вхождений: 25142
Токен: and, количество вхождений: 21134
Токен: of, количество вхождений: 18622
Токен: a, количество вхождений: 16863
Токен: in, количество вхождений: 16232
Токен: ?, количество вхождений: 13730
Токен: #coronavirus, количество вхождений: 12587


**Ответ:** теперь в топ 10 появились знаки препинания

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [16]:
from string import punctuation
noise = stopwords.words('english') + list(punctuation)
words = [el.lower() for el in stopwords.words('english')] # на всякий случай приводим к нижнему регистру (например, чтобы учесть стоп-слово I)

for el in noise:
  del token_counts[el]

print_most_common_10(token_counts)

Токен: #coronavirus, количество вхождений: 12587
Токен: â, количество вхождений: 10498
Токен: , количество вхождений: 10361
Токен: 19, количество вхождений: 10142
Токен: covid, количество вхождений: 8832
Токен: prices, количество вхождений: 6644
Токен: food, количество вхождений: 6213
Токен: , количество вхождений: 6190
Токен: store, количество вхождений: 5494
Токен: supermarket, количество вхождений: 5435


**Ответ:** да, есть. Видимо, символы, которые не входят в ascii. Может, это изначально были какие-то смайлы или другие нетекстовые символы. Также почему-то встречается буква расширенного латинского алфавита â. Возможно, эта буква используется в виде предлога или частицы во французском языке

Удалите из словаря токены из одного символа, с позицией в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [17]:
syms = []
for el in token_counts:
  if len(el) == 1 and ord(el) >= 128:
    syms.append(el)

print(syms)

for el in syms:
  del token_counts[el]

['\x82', 'â', '\x92', 'ã', '\x93', '\x94', '\x97', '\x95', '£', '\x91', '\x83', '¶', '«', '»', '\x96', '\x80', '©', '¼', '¤', '±', '¢', '¡', '½', '\xad', '¨', '·', '³', '²', '°', '\x99', '¸', '\x9f', '§', '´', '¹', '\x98', '¥', '¯', '\x9a', '¦', '\x84', '®', '\x87', '\x89']


In [18]:
print_most_common_10(token_counts)

Токен: #coronavirus, количество вхождений: 12587
Токен: 19, количество вхождений: 10142
Токен: covid, количество вхождений: 8832
Токен: prices, количество вхождений: 6644
Токен: food, количество вхождений: 6213
Токен: store, количество вхождений: 5494
Токен: supermarket, количество вхождений: 5435
Токен: grocery, количество вхождений: 4959
Токен: people, количество вхождений: 4902
Токен: #covid19, количество вхождений: 3726


In [19]:
least_20_common_tokens(token_counts)

Токен: poss, количество вхождений: 1
Токен: https://t.co/binca9vp8p, количество вхождений: 1
Токен: #confinementgeneral, количество вхождений: 1
Токен: https://t.co/zrlg0z520j, количество вхождений: 1
Токен: litteraly, количество вхождений: 1
Токен: #coronavirusfrance, количество вхождений: 1
Токен: https://t.co/usmualq72n, количество вхождений: 1
Токен: @tim_dodson, количество вхождений: 1
Токен: https://t.co/cfxch7a2lu, количество вхождений: 1
Токен: civics, количество вхождений: 1
Токен: https://t.co/iefdnehgdo, количество вхождений: 1
Токен: https://t.co/kw91zj5o5i, количество вхождений: 1
Токен: #govindia, количество вхождений: 1
Токен: #horningsea, количество вхождений: 1
Токен: https://t.co/lsgrxxhjhh, количество вхождений: 1
Токен: https://t.co/8ywakfjexc, количество вхождений: 1
Токен: adara, количество вхождений: 1
Токен: https://t.co/pna797jdkv, количество вхождений: 1
Токен: https://t.co/dqox6usihz, количество вхождений: 1
Токен: https://t.co/9idzsis5oq, количество вхождени

**Ответ:** В топ10 увеличилось число вхождений хештегов, а также единое слово 'covid-19' разбилось на два слова ('covid' и '19'), которые стали частью топа, вытеснив слово 'consumer' из подборки. В топ20 убрались слова со стилистическими ошибками, стало побольше ссылок

Выведите топ-10 популярных хештегов с количеством встреч. Что можно сказать о них?

In [20]:
def print_most_common_10_with_prefix(tokens: list, prefix: str):
    """
    Функция для вывода 10 самых популярных токенов, начинающихся с заданного префикса.
    """
    from collections import Counter
    filtered_tokens = [el for el in tokens if el.startswith(prefix)]
    token_counts = Counter(filtered_tokens)
    for token, count in token_counts.most_common(10):
        print(f'Токен: {token}, Количество вхождений: {count}')

In [21]:
print_most_common_10_with_prefix(tokens, '#')

Токен: #coronavirus, Количество вхождений: 12587
Токен: #covid19, Количество вхождений: 3726
Токен: #covid_19, Количество вхождений: 2525
Токен: #covid2019, Количество вхождений: 1370
Токен: #toiletpaper, Количество вхождений: 1070
Токен: #covid, Количество вхождений: 919
Токен: #socialdistancing, Количество вхождений: 701
Токен: #coronacrisis, Количество вхождений: 627
Токен: #pandemic, Количество вхождений: 359
Токен: #coronaviruspandemic, Количество вхождений: 344


**Ответ:** # -- YOUR ANSWER HERE --

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [22]:
print_most_common_10_with_prefix(tokens, 'https://t.co')

Токен: https://t.co/oxa7swtond, Количество вхождений: 6
Токен: https://t.co/g63rp042ho, Количество вхождений: 5
Токен: https://t.co/r7sagojsjg, Количество вхождений: 4
Токен: https://t.co/wrlhyzizaa, Количество вхождений: 4
Токен: https://t.co/ymsemlvttd, Количество вхождений: 4
Токен: https://t.co/3kfuiojxep, Количество вхождений: 4
Токен: https://t.co/oi39zsanq8, Количество вхождений: 4
Токен: https://t.co/6yvykiab2c, Количество вхождений: 4
Токен: https://t.co/xpcm2xkj4o, Количество вхождений: 4
Токен: https://t.co/gu6b4xpqp4, Количество вхождений: 4


**Ответ:** Популярность, очевидно, несравнима. Ссылка об информации на конкретную страницу может быть полезной, если мы знаем, что содержится на той странице. Например, если это ссылка на какой-то негативный твит, то информация об этом может помочь в классификации комментария (например, если пользователь ведет дискуссию и отвечает на негативные сообщения, то его сообщение будет иметь скорее негативный окрас), но это несколько необычный случай, так что скорее ссылки будут бесполезны

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для изначального выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа, с позицией в таблице Unicode 128 и более и ссылки на t.co



In [23]:
def custom_tokenizer(text: str):
  """
  Функция для токенизации текста с удалением шума.
  """
  from nltk.tokenize import TweetTokenizer
  from collections import Counter
  import nltk
  #nltk.download('stopwords')
  from nltk.corpus import stopwords
  from string import punctuation

  noise = stopwords.words('english') + list(punctuation)
  tokens = TweetTokenizer().tokenize(text.lower())

  for el in tokens:
    if (len(el) == 1 and ord(el) >= 128) or el.startswith('https://t.co'):
      noise.append(el)

  for el in noise:
    if el in tokens:
      tokens.remove(el)

  return tokens


In [45]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '!', '!', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=custom_tokenizer)
cv.fit(train['OriginalTweet'].tolist())

print(len(cv.vocabulary_))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


45816


**Ответ:** с помощью CountVectorizer размер словаря получился меньше

Посмотрим на какой-нибудь конкретный твитт:

In [26]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

('Products that are shelf-stable &amp; long-lived are in demand as consumers are stockpiling staples in anticipation of state- or self-imposed quarantines. Interest in fresh &amp; artisanal foods is being tested as consumers turn to preserved, shelf-stable products https://t.co/MdDEFzqI39',
 1)

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [27]:
tweet = train.iloc[ind]['OriginalTweet']

tweet_vector = cv.transform([tweet])
tweet_matrix = tweet_vector.toarray()

# Находим самый важный токен (с максимальным значением)
most_important_token = cv.get_feature_names_out()[tweet_matrix.argmax()]

# Находим самый неважный токен (с минимальным значением, отличным от 0)
non_zero_indices = tweet_matrix.nonzero()[0]
if len(non_zero_indices) > 0:
    least_important_token_index = tweet_matrix[non_zero_indices].argmin()
    least_important_token = cv.get_feature_names_out()[non_zero_indices[least_important_token_index]]
else:
    least_important_token = None

print(f"Самый важный токен: {most_important_token}")
print(f"Самый неважный токен: {least_important_token}")

Самый важный токен: are
Самый неважный токен: !


**Ответ:** токены определились не совсем хорошо, так как один из них является вспомогательным глаголом, который, по сути, не несёт никакой смысловой нагрузки, а второй является знаком препинания. Проблема в том, что мы использовали CountVectorizer, который просто подсчитывает частоту слов (токенов) в тексте, так что в его контексте "самый важный" - это самый часто встречающийся, а самый неважный - самый редко встречающийся, что не всегда коррелирует с реальностью, как и в данном примере

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer


tv = TfidfVectorizer(tokenizer=custom_tokenizer)
X = tv.fit(train['OriginalTweet'])

tweet_vector = tv.transform([tweet])
tweet_matrix = tweet_vector.toarray()

# Находим самый важный токен (с максимальным TF-IDF значением)
most_important_token = tv.get_feature_names_out()[tweet_matrix.argmax()]

# Находим самый неважный токен (с минимальным TF-IDF значением, отличным от 0)
non_zero_indices = tweet_matrix.nonzero()[0]
if len(non_zero_indices) > 0:
    least_important_token_index = tweet_matrix[non_zero_indices].argmin()
    least_important_token = tv.get_feature_names_out()[non_zero_indices[least_important_token_index]]
else:
    least_important_token = None

print(f"Самый важный токен: {most_important_token}")
print(f"Самый неважный токен: {least_important_token}")

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Самый важный токен: #marketingstrategy
Самый неважный токен: !


**Ответ:** в данном случае токены уже определились лучше, так как TfidfVectorizer учитывает относительную частоту встречаемости слова в данном тексте и во всем корпусе (чем чаще данное слово встречается в данном тексте и чем реже в остальных, тем важнее оно для этого текста), что дает больше информации о важности этого слова для определения тональности

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [59]:
train[train['OriginalTweet'].apply(lambda x: 'amazing' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
17122,24639,69591,"London, England",25-03-2020,Coronavirus Hackney Foodbank up against it a...,1
10851,16963,61915,"Battersea, London",21-03-2020,Wow Edging up to 500 on first day of raising ...,1
25779,35377,80329,Based @1871Chicago,07-04-2020,With people hunkering down it's that more impo...,1
9066,14803,59755,"Denver, CO",20-03-2020,@GovofCO Is this something we can do for our a...,1
11878,18213,63165,"Sydney, New South Wales",21-03-2020,Wow who would've ever thought supermarket shel...,1
...,...,...,...,...,...,...
19919,28079,73031,"Newbury, Berkshire, UK",26-03-2020,The amazing and have managed to get production...,1
7243,12595,57547,Unknown,20-03-2020,@TheLastLeg #IsItOk that this how I imagine it...,1
25263,34743,79695,2828 Dundas St W Toronto,06-04-2020,Sending a shipment ?? of our hand #sanitizer t...,1
31215,42196,87148,United Kingdom,11-04-2020,I hope that after this awful #coronavirus pand...,1


In [67]:
tweet = train.loc[10851]['OriginalTweet']
print(tweet)
tweet_vector = tv.transform([tweet])
tweet_matrix = tweet_vector.toarray()

# Находим самый важный токен (с максимальным TF-IDF значением)
most_important_token = tv.get_feature_names_out()[tweet_matrix.argmax()]

# Находим самый неважный токен (с минимальным TF-IDF значением, отличным от 0)
non_zero_indices = tweet_matrix.nonzero()[0]
if len(non_zero_indices) > 0:
    least_important_token_index = tweet_matrix[non_zero_indices].argmin()
    least_important_token = tv.get_feature_names_out()[non_zero_indices[least_important_token_index]]
else:
    least_important_token = None

print(f"Самый важный токен: {most_important_token}")
print(f"Самый неважный токен: {least_important_token}")

Wow Edging up to  500 on first day of raising funds for This is truly amazing and will go to a refrigerated VAN to deliver surplus food sharing it with the people of The team is in much demand due to 19
Самый важный токен: edging
Самый неважный токен: !


**Ответ:** в данном контексте слово 'edging' используется в положительной тональности, так как речь идет об увеличении суммы привлеченный денег

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [69]:
cv = CountVectorizer(tokenizer=custom_tokenizer)
X_count_train = cv.fit_transform(train['OriginalTweet'])
X_count_test = cv.transform(test['OriginalTweet'])

tv = TfidfVectorizer(tokenizer=custom_tokenizer)
X_tfidf_train = tv.fit_transform(train['OriginalTweet'])
X_tfidf_test = tv.transform(test['OriginalTweet'])

y_train = train['Sentiment']
y_test = test['Sentiment']

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


clf_count = LogisticRegression(max_iter=1000)
clf_count.fit(X_count_train, y_train)


clf_tfidf = LogisticRegression(max_iter=1000)
clf_tfidf.fit(X_tfidf_train, y_train)


y_pred_train_count = clf_count.predict(X_count_train)
y_pred_test_count = clf_count.predict(X_count_test)


y_pred_train_tfidf = clf_tfidf.predict(X_tfidf_train)
y_pred_test_tfidf = clf_tfidf.predict(X_tfidf_test)


train_accuracy_count = accuracy_score(y_train, y_pred_train_count)
test_accuracy_count = accuracy_score(y_test, y_pred_test_count)


train_accuracy_tfidf = accuracy_score(y_train, y_pred_train_tfidf)
test_accuracy_tfidf = accuracy_score(y_test, y_pred_test_tfidf)

print(f"CountVectorizer - Train Accuracy: {train_accuracy_count}")
print(f"CountVectorizer - Test Accuracy: {test_accuracy_count}")
print(f"TfidfVectorizer - Train Accuracy: {train_accuracy_tfidf}")
print(f"TfidfVectorizer - Test Accuracy: {test_accuracy_tfidf}")

CountVectorizer - Train Accuracy: 0.9860743272105937
CountVectorizer - Test Accuracy: 0.8739286426151086
TfidfVectorizer - Train Accuracy: 0.9223408799658266
TfidfVectorizer - Test Accuracy: 0.8524018337651983


**Ответ:** CountVectorizer показал лучший результат. Модели получились довольно хорошими с точки зрения метрики accuracy, но для полноты картины следует ещё сравнить метрики recall

Дальше не делал.

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
  # -- YOUR CODE HERE --
  return tokens

In [ ]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `berplot` содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на оспнове целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 8 Хорошее качество (Бонус 1 балл)

Добейтесь accuracy=0.9 на тестовой выборке (можно сменить токенайзер, векторайзер, модель и т.д.)

In [ ]:
# -- YOUR CODE HERE --